## Comparing Sourmash to RGI for recovering resistance genes
RGI is the current tool used for identifying antimicrobial resistance genes. Relies on read mapping and is mem intensive and slow. Can we get an equally ok result using sourmash instead? Would like for it to work at a scale of 100, or larger. 

For now, I am just comparing raw read rgi (rgi-bwt) to sourmash on the sketched raw reads. 
And using 5 metagenomes

In [ ]:
#  symlink dbs to this directory (in resources)
ln -s /home/amhorst/databases/CARD .
ln -s /home/amhorst/databases/CARD_smash/ .

In [ ]:
# RGI itself only uses the protein homolog model, not all others. Unless specified. 
# Make zip file with only homolog
sourmash sketch dna \
nucleotide_fasta_protein_homolog_model.fasta \
--singleton -p k=18,k=21,k=31,k=51,scaled=100 \
-o ../CARD_smash/card_protein_homolog_model.nucleotide.zip

In [ ]:
# concat all nucleotide card into one fasta file
cat nucleotide_fasta*.fasta > nucleotide_card.fasta

# sketch the CARD db into kmers (maybe later more ksizes)
sourmash sketch dna \
nucleotide_card.fasta \
-p k=18,k=21,k=31,k=51,scaled=100 \
-o nucleotide_card.zip \
--singleton

In [ ]:
# running RGI on the raw reads (in snakefile)
# Think I need to load the db files in the folder where snakefile is
# Can probs do it in snakefile (too lazy rn)
# load the database and specify that it needs a local copy in current folder
rgi load --card_json /home/amhorst/databases/CARD/card.json --local
rgi card_annotation -i /home/amhorst/databases/CARD/card.json > card_annotation.log

# load the json file, and specify where the card db is. 
rgi load -i /home/amhorst/databases/CARD/card.json --card_annotation card_database_v3.3.0.fasta --local

In [ ]:
# try snakefile for 1
srun --account=ctbrowngrp -p med2 -J rgi -t 2:00:00 -c 30 --mem=50gb --pty bash

snakemake --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 30 --rerun-incomplete -k -n
